In [ ]:
from utils import *
from plotly.subplots import make_subplots
from tqdm.auto import tqdm
import json
from loaders import *

In [ ]:
res_s_df, scalars_s = sac_alpha_search_loader()
res_e_df, scalars_e = sac_ent_sched2_loader()
res_b_df, scalars_b = baseline_loader()

In [ ]:
envs = res_s_df["env"].unique()

fig = make_subplots(
    cols=len(envs),
    column_titles=[*envs],
    shared_xaxes=True,
)

colors = make_color_iter()
colors = [next(colors) for _ in range(2)]

axis = 1
for col, env in enumerate(envs, 1):
    df = res_s_df[res_s_df["env"] == env].copy()
    alphas = []
    for _, row in df.iterrows():
        cfg = load_config(row["path"])
        alphas.append(config_value(cfg, "rl.sac.alpha.value"))
    df["alpha"] = alphas

    xpr = px.scatter(df, x="alpha", y="score", trendline="ols")
    for trace in xpr.data:
        fig.add_trace(trace, row=1, col=col)
    fig.update_layout(
        **{f"xaxis{col}": dict(type="log")},
    )

    fig.update_layout(**{f"xaxis{axis}": dict(title="Value of alpha")}),
    axis += 1

fig.update_layout(
    width=1000,
    height=400,
    yaxis_title="Score",
)
fig.write_image("../tex/assets/sac.alpha_perf.pdf")
fig

In [ ]:
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm

envs = res_s_df["env"].unique()

fig = make_subplots(
    cols=len(envs),
    column_titles=[*envs],
    shared_xaxes=True,
)

colors = make_color_iter()
colors = [next(colors) for _ in range(2)]

axis = 1
for col, env in enumerate(envs, 1):
    df = res_s_df[res_s_df["env"] == env].copy()
    alphas, entropies = [], []
    for _, row in df.iterrows():
        cfg = load_config(row["path"])
        alphas.append(config_value(cfg, "rl.sac.alpha.value"))
        test_df = scalars_s.read(row["path"])
        ents = test_df[test_df["tag"] == "rl/entropy"]
        entropies.append(ents.iloc[-1]["value"])
    df["alpha"] = alphas
    df["entropy"] = entropies

    if env == "MsPacman":
        opts = {}
    else:
        opts = dict(log_y=True)

    xpr = px.scatter(
        df,
        x="alpha",
        y="entropy",
        trendline="ols",
        trendline_options=opts,
    )
    for trace in xpr.data:
        fig.add_trace(trace, row=1, col=col)

    fig.update_layout(
        **{
            f"xaxis{col}": dict(type="log"),
            f"yaxis{col}": dict(type="log"),
        },
    )

    fig.update_layout(**{f"xaxis{axis}": dict(title="Value of alpha")}),
    axis += 1

fig.update_layout(
    width=1000,
    height=400,
    yaxis_title="Entropy",
)
fig.write_image("../tex/assets/sac.alpha_ent.pdf")
fig

In [ ]:
envs = res_s_df["env"].unique()

fig = make_subplots(
    cols=len(envs),
    column_titles=[*envs],
    shared_xaxes=True,
)

colors = make_color_iter()
colors = [next(colors) for _ in range(2)]

axis = 1
for col, env in enumerate(envs, 1):
    df = res_s_df[res_s_df["env"] == env].copy()
    alphas, entropies = [], []
    for _, row in df.iterrows():
        cfg = load_config(row["path"])
        alphas.append(config_value(cfg, "rl.sac.alpha.value"))
        test_df = scalars_s.read(row["path"])
        ents = test_df[test_df["tag"] == "rl/entropy"]
        entropies.append(ents.iloc[-1]["value"])
    df["alpha"] = alphas
    df["entropy"] = entropies

    fig.add_trace(
        go.Scatter(x=df["entropy"], y=df["score"], mode="markers", showlegend=False),
        row=1,
        col=col,
    )
    fig.update_layout(
        **{
            f"xaxis{col}": dict(type="log"),
        },
    )

    fig.update_layout(**{f"xaxis{axis}": dict(title="Mean policy entropy")}),
    axis += 1

fig.update_layout(
    width=1000,
    height=400,
    yaxis_title="Score",
)
fig.write_image("../tex/assets/sac.ent_perf.pdf")
fig

In [ ]:
envs = res_s_df["env"].unique()

fig = make_subplots(
    cols=len(envs),
    column_titles=[*envs],
    shared_xaxes=True,
)

colors = make_color_iter()
colors = [next(colors) for _ in range(2)]

axis = 1
for col, env in enumerate(envs, 1):
    df1 = res_e_df[res_e_df["env"] == env].copy()
    df1["tag"] = "SAC"
    df2 = res_b_df[(res_b_df["env"] == env) & (res_b_df["ratio"] == 4)].copy()
    df2["tag"] = "Base"
    df = pd.concat([df1, df2])

    fig.add_trace(
        go.Box(
            x=df["tag"],
            y=df["score"],
            sizemode="sd",
            boxmean=True,
            showlegend=False,
        ),
        row=1,
        col=col,
    )

    fig.update_layout(**{f"xaxis{axis}": dict(title="Configuration")}),
    axis += 1

fig.update_layout(
    width=1000,
    height=400,
    yaxis_title="Score",
)
fig.write_image("../tex/assets/sac.ent_sched2.pdf")
fig